# DeFi Transactions

In this notebook, we will compile all of the transaction data in to one file.

In [1]:
import requests
import json
import pandas as pd
import seaborn as sns
import cryptocompare
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from datetime import datetime
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import missingno as msno
from IPython.display import HTML
import matplotlib.dates as md
import time
import math

### Load Transaction Data

We start by loading the csv files for each transaction type.

In [2]:
df_borrows=pd.read_csv('borrows.csv')
df_borrows.head()

,amount,borrowRate,borrowRateMode,onBehalfOf,pool,reserve,timestamp,user,type,reservePriceETH,reservePriceUSD,amountUSD
0,41501629937448700000000,6.274937,Variable,0x94ee9c600870c4199a1af8496eeb3087f2d1c32f,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,DAI,1621340435,0x94ee9c600870c4199a1af8496eeb3087f2d1c32f,borrows,2.852900e+14,0.994804,4.128600e+22
1,7000000000000,2.589628,Variable,0x51346d389ec7adf11c5b96695ef003dbb849af9a,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1622477822,0x51346d389ec7adf11c5b96695ef003dbb849af9a,borrows,3.812835e+14,1.000000,7.000000e+12
2,15000000000,8.802541,Variable,0x416d7f3823bc7d8dbab00056ede0325e52f0a5c2,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1619775984,0x416d7f3823bc7d8dbab00056ede0325e52f0a5c2,borrows,3.611000e+14,1.004339,1.506508e+10
3,8193190386,48.747052,Stable,0x78cbc5e9eb4f96e5ced75819e76634a768b57e07,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1615481632,0x78cbc5e9eb4f96e5ced75819e76634a768b57e07,borrows,5.562201e+14,0.999391,8.188200e+09
4,11000000000,3.225055,Variable,0xbed4dbd30fd3aed29c2d133fddb611f8aa517c6b,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1626914745,0xbed4dbd30fd3aed29c2d133fddb611f8aa517c6b,borrows,4.971100e+14,1.000000,1.100000e+10


In [3]:
df_repays=pd.read_csv('repays.csv')
df_repays.head()

,amount,onBehalfOf,pool,reserve,timestamp,user,type,reservePriceETH,reservePriceUSD,amountUSD
0,1801803410839,0x0bbe643d5d9dd0498d0c9546f728504a4aab78f4,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1616417822,0x0bbe643d5d9dd0498d0c9546f728504a4aab78f4,repay,5.566700e+14,0.995654,1.793972e+12
1,719667575,0x02e76c933b98c46fabd87fb30c14c7a623319bbe,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1611491659,0x02e76c933b98c46fabd87fb30c14c7a623319bbe,repay,7.542830e+14,1.000000,7.196676e+08
2,7040367570000000000000,0xf2b7f6f9e11f65d7a244363dfd32cdb66100fd6a,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,SUSD,1617162703,0xf2b7f6f9e11f65d7a244363dfd32cdb66100fd6a,repay,5.488300e+14,1.005494,7.079051e+21
3,7400043417403378198609,0xc67d4c582cbc48eb3701b9ab9a0ccdbc23e5aebc,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,DAI,1609463063,0xc67d4c582cbc48eb3701b9ab9a0ccdbc23e5aebc,repay,1.360102e+15,1.003491,7.425878e+21
4,121016000000,0x208b82b04449cd51803fae4b1561450ba13d9510,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1621559868,0x208b82b04449cd51803fae4b1561450ba13d9510,repay,3.446828e+14,1.001534,1.212016e+11


In [4]:
df_deposits=pd.read_csv('deposits.csv')
df_deposits.head()

,amount,onBehalfOf,pool,reserve,timestamp,user,type,reservePriceETH,reservePriceUSD,amountUSD
0,5386488056851070,0xf26c81b576eaeeb532634a0ec4256e3bf64db139,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,AAVE,1626855185,0x2801a71605b5e25816235c7f3cb779f4c9dd60ee,deposit,1.290933e+17,242.104921,1.304095e+18
1,7500000000000000000000,0x762b4a03d4d8ea0f81d2a0254770c9df18361051,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,DAI,1618692791,0x762b4a03d4d8ea0f81d2a0254770c9df18361051,deposit,4.221698e+14,1.000616,7.504617e+21
2,27991243002,0x6082fb69b9886b4cc55d23f87510f7c0841dbc3f,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1610390681,0x6082fb69b9886b4cc55d23f87510f7c0841dbc3f,deposit,9.898730e+14,1.000000,2.799124e+10
3,1800000000000000000000,0xfadb1e5913c439e5bde92826a5b820475f58d24c,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,WETH,1627178698,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,deposit,NaN,NaN,NaN
4,8808995839044045057,0xccdd062202b739037cb3023c14809c243905b8cc,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,WETH,1624224687,0x63a3f444e97d14e671e7ee323c4234c8095e3516,deposit,NaN,NaN,NaN


In [5]:
df_redeems=pd.read_csv('redeems.csv')
df_redeems.head()

,amount,onBehalfOf,pool,reserve,timestamp,user,type,reservePriceETH,reservePriceUSD,amountUSD
0,2100000000000,0xef5c8dfb7abaa3acd759d77fb156cea315eb69b8,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1619717013,0xef5c8dfb7abaa3acd759d77fb156cea315eb69b8,redeem,3.653611e+14,0.997056,2.093817e+12
1,67033672657161035402,0xefeb6afa02d5b368005176c9d8d849aa58a98be2,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,CRV,1626670785,0xefeb6afa02d5b368005176c9d8d849aa58a98be2,redeem,7.479100e+14,1.417037,9.498922e+19
2,19999983800,0x63a3f444e97d14e671e7ee323c4234c8095e3516,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1624224687,0x63a3f444e97d14e671e7ee323c4234c8095e3516,redeem,4.439400e+14,1.003846,2.007690e+10
3,20025978731,0x04956efd10b49c1a7dad15e444092faeb521bddc,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1623371599,0x04956efd10b49c1a7dad15e444092faeb521bddc,redeem,4.067300e+14,1.000057,2.002713e+10
4,1500000000000000000000,0xd291328a6c202c5b18dcb24f279f69de1e065f70,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,ENJ,1611177170,0xd291328a6c202c5b18dcb24f279f69de1e065f70,redeem,7.538678e+14,2.371101,3.556651e+21


In [6]:
df_liquidations=pd.read_csv('liquidations.csv')
df_liquidations.head()

,collateralAmount,collateralReserve,pool,principalAmount,principalReserve,timestamp,user,type,priceUSDTCollateral,reservePriceETHPrincipal,reservePriceUSDPrincipal,reservePriceETHCollateral,reservePriceUSDCollateral,amountUSDPincipal,amountUSDCollateral
0,330855192754556269,WETH,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,63917,GUSD,1626124715,0x3424b949dbc5d2d2654cb6fc622c4d723b2e8fb4,liquidation,493380000000000,4.463772e+14,0.904733,NaN,NaN,5.782783e+04,NaN
1,33825732222059557759476,ENJ,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,55144569141,USDT,1619145033,0x41a792401694fc00467207cfeb9b5f09aa2b834a,liquidation,819170533251580,4.568000e+14,1.000000,4.568000e+14,0.557637,5.514457e+10,1.886249e+22
2,6345434466,USDC,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,82079718741270808,YFI,1621319875,0xc6145175d51edfd6383851dc101f5872a79d7829,liquidation,282720000000000,2.068914e+19,73178.919426,2.810000e+14,0.993916,6.006505e+21,6.306830e+09
3,84893205411112175796,WETH,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,117034106538666291406154,BUSD,1614324006,0x01acb3804ba9c42111c6e9c127831eb486ca1ac7,liquidation,682310000000000,6.908300e+14,1.012487,NaN,NaN,1.184955e+23,NaN
4,3749460623379504856196,LINK,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,52067659439,USDC,1621788289,0x01214a235c6767040c8fef6bd33519a6c7ad79ae,liquidation,566833701101973,5.739400e+14,1.012537,8.767150e+15,15.466882,5.272042e+10,5.799247e+22


In [7]:
df_swaps=pd.read_csv('swaps.csv')
df_swaps.head()

,borrowRateModeFrom,borrowRateModeTo,pool,reserve,stableBorrowRate,timestamp,user,variableBorrowRate,type
0,Stable,Variable,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,DAI,8.988390,1613216311,0xd0698d231d4b65b97a3df9c16aafda8d9b0bda41,3.976780,swap
1,Stable,Variable,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,14.096271,1613943756,0x5c2acae2bfbf75b7a2f045dc7f4326bc4f0b048d,9.096271,swap
2,Variable,Stable,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,12.567462,1617270491,0x7f6fa589d48d42fd290a976abd9634c32405ec0b,4.567462,swap
3,Variable,Stable,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,8.996132,1614461983,0xccec96a03dda5c438996701f26bef600366a137b,3.992264,swap
4,Variable,Stable,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,DAI,11.969824,1619275604,0xf8523151211e6cfb2d285b9b92f2ee517722253b,3.939648,swap


### Compile Transaction Data

Next, we merge them all together in to one big DataFrame.

In [8]:
#merge dataframes
df = df_borrows.append(df_repays).append(df_liquidations).append(df_deposits).append(df_redeems).append(df_swaps)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443401 entries, 0 to 999
Data columns (total 27 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   amount                     436170 non-null  object 
 1   borrowRate                 85391 non-null   float64
 2   borrowRateMode             85391 non-null   object 
 3   onBehalfOf                 436170 non-null  object 
 4   pool                       443401 non-null  object 
 5   reserve                    437170 non-null  object 
 6   timestamp                  443401 non-null  int64  
 7   user                       443401 non-null  object 
 8   type                       443401 non-null  object 
 9   reservePriceETH            339733 non-null  float64
 10  reservePriceUSD            339733 non-null  float64
 11  amountUSD                  339733 non-null  float64
 12  collateralAmount           6231 non-null    object 
 13  collateralReserve          6231 

In [9]:
df.head()

,amount,borrowRate,borrowRateMode,onBehalfOf,pool,reserve,timestamp,user,type,reservePriceETH,...,reservePriceETHPrincipal,reservePriceUSDPrincipal,reservePriceETHCollateral,reservePriceUSDCollateral,amountUSDPincipal,amountUSDCollateral,borrowRateModeFrom,borrowRateModeTo,stableBorrowRate,variableBorrowRate
0,41501629937448700000000,6.274937,Variable,0x94ee9c600870c4199a1af8496eeb3087f2d1c32f,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,DAI,1621340435,0x94ee9c600870c4199a1af8496eeb3087f2d1c32f,borrows,2.852900e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7000000000000,2.589628,Variable,0x51346d389ec7adf11c5b96695ef003dbb849af9a,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1622477822,0x51346d389ec7adf11c5b96695ef003dbb849af9a,borrows,3.812835e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15000000000,8.802541,Variable,0x416d7f3823bc7d8dbab00056ede0325e52f0a5c2,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1619775984,0x416d7f3823bc7d8dbab00056ede0325e52f0a5c2,borrows,3.611000e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8193190386,48.747052,Stable,0x78cbc5e9eb4f96e5ced75819e76634a768b57e07,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1615481632,0x78cbc5e9eb4f96e5ced75819e76634a768b57e07,borrows,5.562201e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11000000000,3.225055,Variable,0xbed4dbd30fd3aed29c2d133fddb611f8aa517c6b,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1626914745,0xbed4dbd30fd3aed29c2d133fddb611f8aa517c6b,borrows,4.971100e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finally, we save this DataFrame as a csv file.

In [10]:
df.to_csv("transactions.csv",index=False)